# Understanding primitive inputs and outputs

This page gives an overview of the inputs and outputs of the Qiskit Runtime V2 primitives that execute workloads on IBM Quantum&trade; compute resources. These primitives provide you with the ability to efficiently define vectorized workloads by using a data structure known as a **Primitive Unified Bloc (PUB)**. These PUBs are the fundamental unit of work a QPU needs to execute these workloads.

After being submitted as jobs to the IBM Quantum Platform, they are executed and the data returned is dependent on the format PUBs which were executed, as well as any error mitigation, suppression, or other runtime options that were defined when the primitive was invoked.

## Overview of PUBS

The PUB data structure defines individual workloads to be submitted to a backend for execution on a QPU. It is a tuple that contains four values:
- A single `QuantumCircuit`, which can contain one or more `Parameters`
- A list of one or more observables, which specify the expectation values to estimate. The data can be in any one of the `ObservablesArrayLike` format such as `Pauli`, `SparsePauliOp`, or `str` (not needed for the `Sampler` primitive)
- A collection of parameter values to bind the circuit against $\theta_k$
- (Optionally) a target precision for expectation values to estimate (not needed for the `Sampler` primitive)

When invoking a primitive's [`run()`](../api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2#run) method, the argument expected is a `list` of one or more PUBs, the data of which will depend on the computation being submitted to the backend.

The below code demonstrates an example set of vectorized inputs to the `Estimator` primitive and executes them on an IBM backend as a single `RuntimeJobV2 ` object.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.circuit import Parameter, QuantumCircuit
from qiskit_ibm_runtime import (
    EstimatorV2 as Estimator,
    SamplerV2 as Sampler,
    QiskitRuntimeService,
)
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import Pauli, SparsePauliOp
import numpy as np

# Instantiate runtime service and get
# the least busy backend
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Define a circuit with two parameters.
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.ry(Parameter("a"), 0)
circuit.rz(Parameter("b"), 0)
circuit.cx(0, 1)
circuit.h(0)

# Transpile the circuit
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
transpiled_circuit = pm.run(circuit)
layout = transpiled_circuit.layout


# Now define a sweep over parameter values, where the two axes are over the
# two parameters in the circuit.
params = np.vstack(
    [
        np.linspace(-np.pi, np.pi, 100),
        np.linspace(-4 * np.pi, 4 * np.pi, 100),
    ]
).T

# Define three observables. The inner length-1 lists cause this array of
# observables to have shape (3, 1), rather than shape (3,) if they were
# omitted.
observables = [
    [SparsePauliOp(["XX", "IY"], [0.5, 0.5])],
    [SparsePauliOp("XX")],
    [SparsePauliOp("IY")],
]
# Apply the same layout as the transpiled circuit.
observables = [
    [observable.apply_layout(layout) for observable in observable_set]
    for observable_set in observables
]

# Estimate the expectation value for all 300 combinations of observables
# and parameter values, where the pub result will have shape (3, 100).
#
# This shape is due to our array of parameter bindings having shape
# (100, 2), combined with our array of observables having shape (3, 1).
estimator_pub = (transpiled_circuit, observables, params)

# Instantiate the new estimator object, then run the transpiled circuit
# using the set of parameters and observables.
estimator = Estimator(mode=backend)
job = estimator.run([estimator_pub])
result = job.result()

## Overview of primitive results

Once one or more PUBs are sent to a QPU for execution and a job successfully completes, the data is returned as a `PrimitiveResult` container object accessed by calling the `RuntimeJobV2.result()` method. This data structure contains an iterable list of `PubResult` objects - one for every PUB - and each of these `PubResult` objects then contains the execution results of the PUB that was submitted. The data for each `PubResult` is contained within a `DataBin` container object, the attributes of which are dependent on the structure of the associated PUB, as well as the error mitigation options specified by the primitive that the job was submitted with (for example, [ZNE](./error-mitigation-and-suppression-techniques#zero-noise-extrapolation-zne) or [PEC](./error-mitigation-and-suppression-techniques#probabilistic-error-cancellation-pec)).

However, no matter what runtime options are specified, each `PubResult` for the Estimator primitive contains a list of expectation values (`PubResult.data.evs`) and associated standard deviations (either `PubResult.data.stds` or `PubResult.data.ensemble_standard_error` depending on the `resilience_level` used).

In [23]:
print(f'The result of the submitted job had {len(result)} PUB and has a value:\n {result}\n')
print(f'The associated PubResult of this job has the following data:\n {result[0].data}\n')
print(f'And this DataBin has attributes: {result[0].data.keys()}')

The result of the submitted job had 1 PUB and has a value:
 PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(3, 100), dtype=float64>), stds=np.ndarray(<shape=(3, 100), dtype=float64>), ensemble_standard_error=np.ndarray(<shape=(3, 100), dtype=float64>), shape=(3, 100)), metadata={'shots': 8192, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 64})], metadata={'dynamical_decoupling': {'enable': False, 'sequence_type': 'XX', 'extra_slack_distribution': 'middle', 'scheduling_method': 'alap'}, 'twirling': {'enable_gates': False, 'enable_measure': True, 'num_randomizations': 'auto', 'shots_per_randomization': 'auto', 'interleave_randomizations': True, 'strategy': 'active-accum'}, 'resilience': {'measure_mitigation': True, 'zne_mitigation': False, 'pec_mitigation': False}, 'version': 2})

The associated PubResult of this job has the following data:
 DataBin(evs=np.ndarray(<shape=(3, 100), dtype=float64>), stds=np.ndarray(<shape=(3, 1

### Sampler output


The Sampler primitive outputs job results in a similar format, with the exception that each `DataBin` will contain one or more `BitArray` objects that store the measured probability distribution(s). The number and attribute label for each bit array object depends on how the `ClassicalRegisters` were defined for the circuit being executed. The measurement data from these `BitArrays` can then be processed into a dictionary with key-value pairs corresponding to each bitstring measured (for example, '1011001') and the number of times (or counts) it was measured.

For example, a circuit that has measurement instructions added by the [`QuantumCircuit.measure_all()`](../api/qiskit/qiskit.circuit.QuantumCircuit#measure_all) function possesses a classical register with the label *'meas'*. After execution, a count data dictionary can be created by executing:

In [ ]:
# Add measurement instructions to the example circuit
circuit.measure_all()

# Transpile the circuit
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
transpiled_circuit = pm.run(circuit)

# Create a PUB for the Sampler primitive using the same parameters defined earlier
sampler_pub = (transpiled_circuit, params)


sampler = Sampler(mode=backend)
job = sampler.run([sampler_pub])
result = job.result()

In [ ]:
print(f'Sampler Data is: {result[0].data}')
print(f'It has the key-value pair: {result[0].data.items()}')

result[0].data.meas.get_counts()

Sampler Data is: DataBin(meas=BitArray(<shape=(100,), num_shots=4096, num_bits=2>), shape=(100,))
Its keys are: dict_items([('meas', BitArray(<shape=(100,), num_shots=4096, num_bits=2>))])


{'11': 115808, '10': 104963, '01': 89334, '00': 99495}

## Result Metadata

In addition to the execution results, both the `PrimitiveResult` and `PubResult` objects contain a metadata attribute about the job that was submitted. The metadata that contains information for all submitted PUBs (such as the various [runtime options](../api/qiskit-ibm-runtime/options) available) can be found in the `PrimitiveResult.metatada`, while the metadata specific to each PUB is found in `PubResult.metadata`.

In [24]:
# Print out the results metadata
print(f'The metadata of the PrimitiveResult is:')
for key, val in result.metadata.items():
    print(f"'{key}' : {val},")

print(f'\nThe metadata of the PubResult result is:')
for key, val in result[0].metadata.items():
    print(f"'{key}' : {val},")

The metadata of the PrimitiveResult is:
'dynamical_decoupling' : {'enable': False, 'sequence_type': 'XX', 'extra_slack_distribution': 'middle', 'scheduling_method': 'alap'},
'twirling' : {'enable_gates': False, 'enable_measure': True, 'num_randomizations': 'auto', 'shots_per_randomization': 'auto', 'interleave_randomizations': True, 'strategy': 'active-accum'},
'resilience' : {'measure_mitigation': True, 'zne_mitigation': False, 'pec_mitigation': False},
'version' : 2,

The metadata of the PubResult result is:
'shots' : 8192,
'target_precision' : 0.015625,
'circuit_metadata' : {},
'resilience' : {},
'num_randomizations' : 64,
